In [1]:
import argparse
import gc
import glob
import json
import math
import operator
import os
import random
import re
import sys
import time
import unicodedata
import warnings
from collections import Counter  # Import Counter to count occurrences of n-grams
from datetime import datetime, timedelta
from functools import partial
from glob import glob
from multiprocessing import cpu_count
from pathlib import Path
from typing import (
    Annotated,
    Any,
    Dict,
    List,
    Literal,
    Optional,
    Sequence,
    Tuple,
    TypedDict,
)

import datasets
import evaluate
import fasttext
import gensim
import joblib
import matplotlib.pyplot as plt  # For plotting graphs
import nbformat
import nltk
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go  # Import Plotly for creating visualizations
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import PyPDF2
import requests  # Import the requests library to make HTTP requests
import scipy.sparse
import spacy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import umap
# from bertopic import BERTopic
# from bertopic.representation import MaximalMarginalRelevance
# from bertopic.vectorizers import ClassTfidfTransformer
from bs4 import BeautifulSoup  # Import BeautifulSoup to parse HTML content
from datasets import Dataset, load_dataset
from dotenv import load_dotenv
from IPython.display import Image, display
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.chat_models import ChatOllama, ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.pydantic_v1 import BaseModel, Field
from langchain_chroma import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_community.document_loaders import (
    PyPDFLoader,
    UnstructuredURLLoader,
    YoutubeLoader,
)
from langchain_community.document_loaders.youtube import YoutubeLoader
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.documents import Document
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.graph import CurveStyle
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.tools import StructuredTool
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_openai import ChatOpenAI, OpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import END, START, StateGraph
from langgraph.prebuilt import ToolExecutor, ToolInvocation
from lightgbm import LGBMClassifier
from nltk import word_tokenize  # Import word_tokenize to split text into words
from nltk.corpus import stopwords, wordnet
from nltk.util import ngrams  # Import ngrams to generate n-grams from text
from sentence_transformers import (
    InputExample,
    SentenceTransformer,
    evaluation,
    losses,
    models,
)
from sklearn.calibration import CalibratedClassifierCV
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs, make_classification, make_regression
from sklearn.decomposition import NMF, PCA, TruncatedSVD
from sklearn.ensemble import (
    ExtraTreesClassifier,
    RandomForestClassifier,
    StackingClassifier,
)
from sklearn.exceptions import ConvergenceWarning
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.manifold import TSNE  # Import t-SNE from scikit-learn
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    calinski_harabasz_score,
    classification_report,
    confusion_matrix,
    f1_score,
    make_scorer,
    matthews_corrcoef,
    silhouette_score,
)
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import (
    GridSearchCV,
    KFold,
    RandomizedSearchCV,
    train_test_split,
)
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import (
    Pipeline,  # Pipeline applies a list of transforms sequentially. You can also add an estimator at the end.  # Pipeline applies a list of transforms. You can also add an estimator at the end, so it will be completely encapsulated.
    make_pipeline,
)
from sklearn.preprocessing import (
    FunctionTransformer,  # FunctionTransformer allows applying an arbitrary function to the data, useful for custom transformations.  # FunctionTransformer allows to apply an arbitrary function to the data, so we can use it in the pipeline
)
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from skopt import BayesSearchCV
from spacy import displacy
from spacy.tokens import DocBin
from spacy.training.example import Example
from torch.utils.data import DataLoader
from torchtext import data
from torchtext.data import TabularDataset
from transformers import (
    AutoConfig,
    AutoModel,
    AutoModelForMaskedLM,
    AutoModelForSequenceClassification,
    AutoModelForTokenClassification,
    AutoTokenizer,
    BertConfig,
    BertForSequenceClassification,
    BertModel,
    BertTokenizer,
    DataCollatorForLanguageModeling,
    DataCollatorForTokenClassification,
    DataCollatorWithPadding,
    Pipeline,
    PreTrainedModel,
    PreTrainedTokenizer,
    Trainer,
    TrainingArguments,
    pipeline,
)
from wordcloud import WordCloud
from xgboost import XGBClassifier

from helpers.text import (
    remove_accented_characters,
    remove_excessive_spaces,
    remove_numbers_punctuation_from_text,
    remove_short_words,
)


/home/jacob/.cache/pypoetry/virtualenvs/imd1107-nlp-V6lLOcpZ-py3.12/lib/python3.12/site-packages/PyPDF2/__init__.py:21: DeprecationWarning: PyPDF2 is deprecated. Please move to the pypdf library instead.
  warnings.warn(
/home/jacob/.cache/pypoetry/virtualenvs/imd1107-nlp-V6lLOcpZ-py3.12/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
<frozen importlib._bootstrap>:488: DeprecationWarning: T